## Inputs

In [1]:
data_dir = r"..\..\..\Applications\DSLazy\NDS_UNPACK_DQMJ1\data"

## Code

In [8]:
import io
import pathlib
import tempfile
from dataclasses import dataclass
from typing import Any

import numpy as np
import plotnine as plt9
import pandas as pd

import dqmj1_randomizer.data
from dqmj1_randomizer.randomize.btl_enmy_prm import BtlEnmyPrm, randomize_btl_enmy_prm
from dqmj1_randomizer.randomize.regions import Region
from dqmj1_randomizer.state import Monsters, State

In [9]:
data_dir = pathlib.Path(data_dir)

original_btl_enmy_prm_filepath = data_dir / "BtlEnmyPrm.bin"

encounters_info = pd.read_csv(dqmj1_randomizer.data.data_path / "btl_enmy_prm_info.csv").fillna("n")

In [10]:
def calculate_encounter_type(encounter: dict[str, Any]) -> str:
    if encounter["is_boss"] == "y":
        return "Boss"
    elif encounter["is_scout_monster"] == "y":
        return "Scout Monster"
    elif encounter["is_gift_monster"] == "y":
        return "Gift"
    elif encounter["is_starter"] == "y" or encounter["is_unused_starter"] == "y":
        return "Starter"
    else:
        return "Normal"

def calculate_highest_stat(encounter: dict[str, Any]) -> str:
    stats = [
        ("Attack", encounter["attack"]),
        ("Defense", encounter["defense"]),
        ("Agility", encounter["agility"]),
        ("Wisdom", encounter["wisdom"]),
    ]

    return sorted(stats, key=lambda s: -s[1])[0][0]

def calculate_stat_balance(encounter: dict[str, Any]) -> str:
    stats = [
        encounter["attack"],
        encounter["defense"],
        encounter["agility"],
        encounter["wisdom"],
    ]

    stat_total = sum(stats)
    stat_mean = stat_total / 4.0
    stat_min = min(stats)
    stat_max = max(stats)
    std_dev = np.std(stats)

    return std_dev / stat_mean

In [11]:
def load_btl_enmy_prm(input_stream) -> pd.DataFrame:
    encounters = BtlEnmyPrm.from_bin(input_stream).to_pd()
    
    encounters = encounters.join(encounters_info)
    
    encounters = encounters[encounters["exclude"] != "y"]

    assert len(encounters) > 0
    
    encounters["stat_total"] = encounters["attack"] + encounters["defense"] + encounters["agility"] + encounters["wisdom"]
    encounters["encounter_type"] = [calculate_encounter_type(row) for _, row in encounters.iterrows()]
    encounters["highest_stat"] = [calculate_highest_stat(row) for _, row in encounters.iterrows()]
    encounters["stat_balance"] = [calculate_stat_balance(row) for _, row in encounters.iterrows()]
    
    return encounters

In [12]:
with original_btl_enmy_prm_filepath.open("rb") as input_stream:
    original = load_btl_enmy_prm(input_stream)
original

,species_id,item_drops,gold,exp,level,max_hp,max_mp,attack,defense,agility,...,is_unused_starter,is_gift_monster,is_scout_monster,is_gift_incarnus,exclude,swap_drop,stat_total,encounter_type,highest_stat,stat_balance
1,318,"[{'item_id': 0, 'chance_denominator_2_power': ...",0,0,40,4065,255,336,154,92,...,n,n,n,n,n,n,838,Boss,Attack,0.446824
2,176,"[{'item_id': 0, 'chance_denominator_2_power': ...",300,225,12,235,9,55,40,26,...,n,n,n,n,n,y,155,Boss,Attack,0.273947
3,224,"[{'item_id': 0, 'chance_denominator_2_power': ...",650,1860,18,450,0,85,36,10,...,n,n,n,n,n,y,164,Boss,Attack,0.666394
4,275,"[{'item_id': 0, 'chance_denominator_2_power': ...",478,3016,35,812,100,145,56,82,...,n,n,n,n,n,y,318,Boss,Attack,0.519734
5,84,"[{'item_id': 0, 'chance_denominator_2_power': ...",1050,4620,28,1156,64,187,84,110,...,n,n,n,n,n,y,439,Boss,Attack,0.439553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,168,"[{'item_id': 0, 'chance_denominator_2_power': ...",0,5264,36,1768,255,348,166,126,...,n,n,n,n,n,n,723,Boss,Attack,0.558363
854,200,"[{'item_id': 0, 'chance_denominator_2_power': ...",30,185,13,132,12,105,90,73,...,n,n,n,n,n,n,287,Normal,Attack,0.452841
855,308,"[{'item_id': 0, 'chance_denominator_2_power': ...",0,0,32,486,238,184,138,204,...,n,n,n,n,n,n,722,Boss,Agility,0.141546
856,274,"[{'item_id': 0, 'chance_denominator_2_power': ...",0,0,32,532,100,256,174,160,...,n,n,n,n,n,n,810,Boss,Attack,0.187833


In [23]:
def randomize(seed: int) -> pd.DataFrame:
    state = State(
        original_rom=None,
        seed=seed,
        region=Region.NorthAmerica,
        monsters=Monsters(
            randomize=True,
            include_bosses=True,
            transfer_boss_item_drops=False,
            include_starters=True,
            include_gift_monsters=True,
        ),
    )
    
    output_stream = io.BytesIO()
    with original_btl_enmy_prm_filepath.open("rb") as input_stream:
        randomize_btl_enmy_prm(state=state, input_stream=input_stream, output_stream=output_stream)
    
        randomized = load_btl_enmy_prm(io.BytesIO(output_stream.getvalue()))
    
    return randomized

In [45]:
@dataclass
class Comparison:
    starter_stat_change: int
    normal_encounter_difficulty_increase: int
    normal_encounter_difficulty_change: int

def compare_games(before: pd.DataFrame, after: pd.DataFrame) -> Comparison:
    normal_encounter_stat_changes = after[after["encounter_type"] == "Normal"]["stat_total"] - \
        before[before["encounter_type"] == "Normal"]["stat_total"]

    normal_encounter_difficulty_increase = sum(change if change >= 0 else 0 for change in normal_encounter_stat_changes)
    normal_encounter_difficulty_change = sum(normal_encounter_stat_changes)

    starter_stat_changes = after[after["is_starter"] == "y"]["stat_total"] - \
        before[before["is_starter"] == "y"]["stat_total"]
    starter_stat_change = sum(starter_stat_changes)

    print("======================")
    print(before[before["is_starter"] == "y"][["species_id", "stat_total"]])
    print(after[after["is_starter"] == "y"][["species_id", "stat_total"]])
    
    return Comparison(
        starter_stat_change=starter_stat_change,
        normal_encounter_difficulty_increase=normal_encounter_difficulty_increase,
        normal_encounter_difficulty_change=normal_encounter_difficulty_change,
    )

In [46]:
for seed in range(0, 10):
    randomized = randomize(seed)
    print(compare_games(original, randomized))

    species_id  stat_total
48         240          53
49          96          46
50         144          43
    species_id  stat_total
48         240          70
49          81         582
50         201         626
Comparison(starter_stat_change=1136, normal_encounter_difficulty_increase=53275, normal_encounter_difficulty_change=38136)
    species_id  stat_total
48         240          53
49          96          46
50         144          43
    species_id  stat_total
48         159         702
49         167         914
50          17         492
Comparison(starter_stat_change=1966, normal_encounter_difficulty_increase=51490, normal_encounter_difficulty_change=31301)
    species_id  stat_total
48         240          53
49          96          46
50         144          43
    species_id  stat_total
48           1         177
49          56         508
50         304         815
Comparison(starter_stat_change=1358, normal_encounter_difficulty_increase=61564, normal_encounter_difficul